# 🏛️ Semantic Objects Tutorial

The **Semantic Objects** library provides a simplified Pythonic interface to complex ontologies in the built environment (such as ASHRAE Standard 223). 

### Why use this library?
Traditionally, working with these ontologies requires deep knowledge of **RDF, SHACL, and SPARQL**. This library abstracts those complexities, allowing you to define objects that feed directly into:
* **Model Creation**: Aligning model usage without manual RDF creation.
* **Validation**: Generating SHACL/ontology statements for inference.
* **Querying**: Initializing semantic objects directly from data graphs.

---

In [1]:
# 🛠️ Setup and Imports
from semantic_objects.s223 import *
from semantic_objects.core import export_templates
from semantic_objects.build_model import BMotifSession
from pprint import pprint

# Note: Ensure you have buildingmotif installed
# e.g., pip install buildingmotif[bacnet-ingress]

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


## 🔍 Exploring Entities and Properties

In this library, **Entities** (like a Window) represent physical or logical objects, while **Properties** define the attributes associated with them.

### 1. Inspecting an Entity
An entity is composed of several **Fields**, which represent the semantic properties required by the ontology.

In [2]:
# View the fields associated with a Window (e.g., Area, Tilt, Azimuth)
pprint(Window.__dataclass_fields__)

{'area': Field(name='area',type=<class 'semantic_objects.s223.properties.Area'>,default=<dataclasses._MISSING_TYPE object at 0x101707c10>,default_factory=<dataclasses._MISSING_TYPE object at 0x101707c10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'azimuth': Field(name='azimuth',type=<class 'semantic_objects.s223.properties.Azimuth'>,default=<dataclasses._MISSING_TYPE object at 0x101707c10>,default_factory=<dataclasses._MISSING_TYPE object at 0x101707c10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'tilt': Field(name='tilt',type=<class 'semantic_objects.s223.properties.Tilt'>,default=<dataclasses._MISSING_TYPE object at 0x101707c10>,default_factory=<dataclasses._MISSING_TYPE object at 0x10

### 2. Diving into Property Metadata
Properties are defined with specific values, units, and **Quantity Kinds**.

In [3]:
# Inspect the metadata of the Area property
print(f"Property Class: {Area}")
pprint(Area.__dataclass_fields__)

Property Class: <class 'semantic_objects.s223.properties.Area'>
{'qk': Field(name='qk',type=<class 'semantic_objects.qudt.quantitykinds.QuantityKind'>,default=<class 'semantic_objects.qudt.quantitykinds.Area'>,default_factory=<dataclasses._MISSING_TYPE object at 0x101707c10>,init=False,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': False, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD),
 'unit': Field(name='unit',type=typing.Optional[semantic_objects.units.Unit],default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x101707c10>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True}),kw_only=False,_field_type=_FIELD),
 'value': Field(name='value',type=<class 'float'>,default=<dataclasses._MISSING_TYPE object at 0x101707c10>,default_factory=<dataclasses._MISSING_TYPE object at 0x101707c10>,init=True,repr=True,hash=None,compar

## 📝 Template Generation

A core feature is exporting these objects into **YAML templates** for use by `buildingmotif` or `Semantic_Model_Builder`.

### Exporting to Directory
This retrieves related classes, creates templates, and exports them to a folder.

In [4]:
export_templates(Window, 'templates')

TypeError: issubclass() arg 1 must be a class

### Inspecting the Generated Template
We can see exactly how the library translates the Python object into a semantic template.

In [ ]:
related_classes = get_related_classes(Window)
print('Example generated YAML template for Tilt:')
print(related_classes[-1][0].to_yaml())

Example generated YAML template for Tilt:
Window:
  body: |+
    @prefix P: <urn:___param___#> .
    @prefix s223: <http://data.ashrae.org/standard223#> .

    P:name a s223:Window ;
        s223:hasProperty P:area,
            P:azimuth,
            P:tilt .

  dependencies:
  - args:
      name: area
    template: Area
  - args:
      name: azimuth
    template: Azimuth
  - args:
      name: tilt
    template: Tilt



In [ ]:
related_classes

([semantic_objects.s223.relations.hasProperty,
  semantic_objects.s223.relations.hasQuantityKind,
  semantic_objects.s223.relations.hasValue,
  semantic_objects.s223.relations.hasUnit],
 [semantic_objects.s223.entities.Window,
  semantic_objects.s223.properties.Area,
  semantic_objects.s223.properties.Tilt,
  semantic_objects.s223.properties.Azimuth,
  semantic_objects.units.Unit,
  semantic_objects.qudt.quantitykinds.QuantityKind])

## ⚖️ Validation and SHACL Generation

The library can generate **RDF class definitions** and **SHACL shapes** to ensure your models adhere to ontology constraints.

In [ ]:
# Generate RDF definition including the parent hierarchy (RDFS inheritance)
print("--- Full Hierarchy ---")
print(Space.generate_rdf_class_definition(include_hierarchy=True))

# Generate just the local class constraints
print("--- Local Constraints Only ---")
print(Space.generate_rdf_class_definition(include_hierarchy=False))

--- Full Hierarchy ---
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

s223:Space a s223:Class,
        rdfs:Class,
        sh:NodeShape ;
    rdfs:label "Domain Space" ;
    rdfs:subClassOf s223:DomainSpace ;
    sh:property [ a sh:PropertyShape ;
            rdfs:comment "If the relation `encloses` is present it must associate the `Space` with a `DomainSpace`." ;
            sh:class s223:DomainSpace ;
            sh:message "s223: If the relation `encloses` is present it must associate the `Space` with a `DomainSpace`." ;
            sh:path s223:encloses ],
        [ a sh:PropertyShape ;
            rdfs:comment "If the relation `contains` is present it must associate the `Space` with a `PhysicalSpace`." ;
            sh:class s223:PhysicalSpace ;
            sh:message "s223: If the relation `contains` is present it must asso

In [ ]:
print(Space.get_sparql_query())

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX s223: <http://data.ashrae.org/standard223#>
SELECT DISTINCT * WHERE { ?name s223:hasProperty ?area .
?name rdf:type s223:Space .
?area rdf:type s223:Area . }


In [ ]:
BMotifSession = BMotifSession()
BMotifSession.load_class(Space)


DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage


DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasProperty'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'bf55870e-d678-4184-8bc4-1986cd0629c3' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasQuantityKind'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '744a4cc1-9b75-42b6-abd5-df8ec6f13800' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasValue'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '2c7a3bfc-e738-46e5-879d-45cbc3a39858' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hasUnit'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'eb9316

In [ ]:
Space._get_template_parameters()

{'area': Field(name='area',type=<class 'semantic_objects.s223.properties.Area'>,default=<dataclasses._MISSING_TYPE object at 0x100b439d0>,default_factory=<dataclasses._MISSING_TYPE object at 0x100b439d0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD)}

In [ ]:
# Should optionally take a name input 
s = Space(area=100)

In [ ]:
s._get_evaluation_dict()

{'area': None}

In [ ]:
s._get_template_parameters()

{'area': Field(name='area',type=<class 'semantic_objects.s223.properties.Area'>,default=<dataclasses._MISSING_TYPE object at 0x100b439d0>,default_factory=<dataclasses._MISSING_TYPE object at 0x100b439d0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'relation': None, 'min': 1, 'max': None, 'qualified': True, 'label': None, 'comment': None}),kw_only=False,_field_type=_FIELD)}

In [ ]:
s

Space(area=100)